In [1]:
import pandas as pd
import numpy as np
import glob
from ast import literal_eval

pd.set_option("display.max_columns", 20)
pd.set_option("display.max_rows", 2000)

### All the functions (Need to put these into a separate .py file)

In [2]:
def read_and_merge(path):
    '''
    Input: Path of directory where all the csv files are located
    Output: Merged dataframe of all csv files in the directory
    '''
    ## Finds all the csv files in a directory and put them into a list
    df_list = []
    for file in glob.glob(path):
        df = pd.read_csv(str(file))
        df_list.append(df)
        
    ## Merges all dataframes in the csv files
    df_merged = df_list[0]
    for df in df_list[1:]:
        df_merged = pd.concat([df_merged, df])
        
    return df_merged

In [3]:
def df_preprocessing_pipeline(df):
    '''
    Input: Datafrome to be preprocessed
    Output: Preprocessed dataframe
    '''
    ## Drops rows with NaNs in the 'Transcript' column
    df = df[df['Transcript'].notna()]
    
    ## Drops duplicate videos
    df = df.drop_duplicates(subset=['Video_ID'])
    
    ## Drops videos with duration < 1 minute
    df_cleaned = df[(df['Duration'].str.len() >=5) | (df['Duration'].str[-4]!='0')]
    
    return df_cleaned
    

In [4]:
def grab_transcript_text(df):
    '''
    Input: Dataframe with raw values in the Transcript column
    Output: Dataframe with only text in the Transcript column, in a string format
    '''
    ## Create a function for grabbing the text values and putting into a single string
    def transcript_text_string(transcript_dict_list):
        text_list = []
        for dict_text in literal_eval(transcript_dict_list):
            text = dict_text['text']
            text_list.append(text)
            
        text_string = ' '.join(text_list)
            
        return text_string
    
    df['Transcript'] = df['Transcript'].apply(transcript_text_string)
            
    return df   

In [11]:
def df_to_csv(df_str):
    '''
    Input: Name of a dataframe in a string format
    Output: CSV file of the dataframe saved into the Data folder
    '''
    eval(df_str).to_csv('../Data/{}.csv'.format(df_str), index=False)

### Read in the CSVs and merge them

In [5]:
df_videos_raw = read_and_merge('/Users/mike/Desktop/GitHub Repositories/project5_youtube_recommender/Data/*.csv')

### Preprocess the dataframe to remove rows with NaNs, duplicate vidoes, videos with duration < 1 minute, and irrelevant videos not related to investing

In [6]:
df_videos_cleaned_v1 = df_preprocessing_pipeline(df_videos_raw).reset_index(drop=True)

In [7]:
## Drop irrelevant videos not related to stock investing
rows_to_drop = df_videos_cleaned_v1[(df_videos_cleaned_v1['Title'].str.contains('Forex')==True) |
                            (df_videos_cleaned_v1['Title'].str.contains('Crypto')==True) |
                            (df_videos_cleaned_v1['Title'].str.contains('Brand Moat')==True) |
                            (df_videos_cleaned_v1['Title'].str.contains('Richard Moat')==True) |
                            (df_videos_cleaned_v1['Title'].str.contains('Clips From The Moat')==True) |
                            (df_videos_cleaned_v1['Title'].str.contains('Geraghty-Moats')==True) |
                            (df_videos_cleaned_v1['Title'].str.contains('Bitcoin')==True) |
                            (df_videos_cleaned_v1['Video_ID']=='gx6JOPTWznQ') | 
                            (df_videos_cleaned_v1['Video_ID']=='drGxSXK8Jwc') | 
                            (df_videos_cleaned_v1['Video_ID']=='u7KkCSpuBLo') |
                            (df_videos_cleaned_v1['Video_ID']=='wM7uNy_ColA') |
                            (df_videos_cleaned_v1['Video_ID']=='OEUWbKSXYD0') |
                            (df_videos_cleaned_v1['Video_ID']=='UDRgo-FLGh0') |
                            (df_videos_cleaned_v1['Video_ID']=='571EoMTCUSw') |
                            (df_videos_cleaned_v1['Video_ID']=='CV39QzFpJx4') |
                            (df_videos_cleaned_v1['Video_ID']=='Mak8d_JSgyU') |
                            (df_videos_cleaned_v1['Video_ID']=='YUX3Y8ONuB4') |
                            (df_videos_cleaned_v1['Video_ID']=='WyVnrJq46Iw') |
                            (df_videos_cleaned_v1['Video_ID']=='I4AXdn1w15c') |
                            (df_videos_cleaned_v1['Video_ID']=='hnISD7EKV-A') |
                            (df_videos_cleaned_v1['Video_ID']=='PdTBLPL2yos') |
                            (df_videos_cleaned_v1['Video_ID']=='YFNjyIcnJSw') |
                            (df_videos_cleaned_v1['Video_ID']=='b68xiEj9iyc') |
                            (df_videos_cleaned_v1['Video_ID']=='2wcnuRSMURw') |
                            (df_videos_cleaned_v1['Video_ID']=='HL-DFblqDxc') |
                            (df_videos_cleaned_v1['Video_ID']=='TzWh3hBJZEs') |
                            (df_videos_cleaned_v1['Video_ID']=='PzWIch-kh3c') |
                            (df_videos_cleaned_v1['Video_ID']=='XXsoswLThAI') |
                            (df_videos_cleaned_v1['Video_ID']=='7WL1Vw4g8OA') |
                            (df_videos_cleaned_v1['Video_ID']=='DCkyNT39TUA') |
                            (df_videos_cleaned_v1['Video_ID']=='NC2HECcxYjk') |
                            (df_videos_cleaned_v1['Video_ID']=='oQx2TNvnO7Q') |
                            (df_videos_cleaned_v1['Video_ID']=='x7npmocd2Gk') |
                            (df_videos_cleaned_v1['Video_ID']=='N7t7sI8UPDI') |
                            (df_videos_cleaned_v1['Video_ID']=='AQRLlMrwo7U') |
                            (df_videos_cleaned_v1['Video_ID']=='5FtSC0RqmfQ') |
                            (df_videos_cleaned_v1['Video_ID']=='S_h2mHRe_hE') |
                            (df_videos_cleaned_v1['Video_ID']=='9QBoh4U5WC4') |
                            (df_videos_cleaned_v1['Video_ID']=='u-JlA-fk_xM') |
                            (df_videos_cleaned_v1['Video_ID']=='u7RO3Eeu1PI') |
                            (df_videos_cleaned_v1['Video_ID']=='_grMQithgA0') |
                            (df_videos_cleaned_v1['Video_ID']=='ug2_GWkbF1o') |
                            (df_videos_cleaned_v1['Video_ID']=='brtr9vSxh7M') |
                            (df_videos_cleaned_v1['Video_ID']=='fyi9g5AkBkY') |
                            (df_videos_cleaned_v1['Video_ID']=='YDZzvisSsvQ') |
                            (df_videos_cleaned_v1['Video_ID']=='79GibY7iTZg') |
                            (df_videos_cleaned_v1['Video_ID']=='oYU89WNmH8g') |
                            (df_videos_cleaned_v1['Video_ID']=='PESfQZiv5KQ') |
                            (df_videos_cleaned_v1['Video_ID']=='PK5b6Zd7thg') |
                            (df_videos_cleaned_v1['Video_ID']=='9_nf0zaLy2Y') |
                            (df_videos_cleaned_v1['Video_ID']=='hw0KmIZi2g8') |
                            (df_videos_cleaned_v1['Video_ID']=='CUCzCVa_7cY') |
                            (df_videos_cleaned_v1['Video_ID']=='lbOvcKqEdU0') |
                            (df_videos_cleaned_v1['Video_ID']=='xy0aNUVLPpY') |
                            (df_videos_cleaned_v1['Video_ID']=='YV9e6tYWgIU') |
                            (df_videos_cleaned_v1['Video_ID']=='sF0hIo0VAuk') |
                            (df_videos_cleaned_v1['Video_ID']=='CTTKFUzeWI0') |
                            (df_videos_cleaned_v1['Video_ID']=='ts-TptT_NZI') |
                            (df_videos_cleaned_v1['Video_ID']=='iHxAPsmos4M') |
                            (df_videos_cleaned_v1['Video_ID']=='IFYrFeAygfc') |
                            (df_videos_cleaned_v1['Video_ID']=='LgXL6q7vdm8') |
                            (df_videos_cleaned_v1['Video_ID']=='TwGAQ2tuJPg') |
                            (df_videos_cleaned_v1['Video_ID']=='W0kzME-kSFk') |
                            (df_videos_cleaned_v1['Video_ID']=='Q6e6oCXbbPo') |
                            (df_videos_cleaned_v1['Video_ID']=='kuiUq_TCRJ0') |
                            (df_videos_cleaned_v1['Video_ID']=='M_SiX-ASntY') |
                            (df_videos_cleaned_v1['Video_ID']=='xjE311nBsXg') |
                            (df_videos_cleaned_v1['Video_ID']=='ejn--JYXsNU') |
                            (df_videos_cleaned_v1['Video_ID']=='KYePUROUoAs') |
                            (df_videos_cleaned_v1['Video_ID']=='jJjtYb-FhYU') |
                            (df_videos_cleaned_v1['Video_ID']=='kebonpz4bD0') |
                            (df_videos_cleaned_v1['Video_ID']=='_gm8YzjDkeI') |
                            (df_videos_cleaned_v1['Video_ID']=='k6xfTJzoEtY') |
                            (df_videos_cleaned_v1['Video_ID']=='CgOfNkP8kJo') |
                            (df_videos_cleaned_v1['Video_ID']=='rW9P5EyiiMU') |
                            (df_videos_cleaned_v1['Video_ID']=='2MQ46Es9rTU') |
                            (df_videos_cleaned_v1['Video_ID']=='KZ9Wf9wP0UM') |
                            (df_videos_cleaned_v1['Video_ID']=='A7yyPn3_18E') |
                            (df_videos_cleaned_v1['Video_ID']=='Wtcz4zi1IK4') |
                            (df_videos_cleaned_v1['Video_ID']=='Y0amF_F6EvI') |
                            (df_videos_cleaned_v1['Video_ID']=='e9KXzTdqV80') |
                            (df_videos_cleaned_v1['Video_ID']=='ViFgKUbGxHo') |
                            (df_videos_cleaned_v1['Video_ID']=='rGKBRacvasg') |
                            (df_videos_cleaned_v1['Video_ID']=='ra9SufwwaRk') |
                            (df_videos_cleaned_v1['Video_ID']=='HCBbJKs5SgA') |
                            (df_videos_cleaned_v1['Video_ID']=='5HOsl9WEvX8') |
                            (df_videos_cleaned_v1['Video_ID']=='Dq1ddyBapRA') |
                            (df_videos_cleaned_v1['Video_ID']=='0FNHMZJ2A0k') |
                            (df_videos_cleaned_v1['Video_ID']=='efQmjeCZiiU') |
                            (df_videos_cleaned_v1['Video_ID']=='kjz1YPA0WTI') |
                            (df_videos_cleaned_v1['Video_ID']=='YAdPCXpwj6I') |
                            (df_videos_cleaned_v1['Video_ID']=='NftVyw7gQw0') |
                            (df_videos_cleaned_v1['Video_ID']=='osy4iN_ae-E') |
                            (df_videos_cleaned_v1['Video_ID']=='ED8x64yOrQY') |
                            (df_videos_cleaned_v1['Video_ID']=='VIoO953WF5c') |
                            (df_videos_cleaned_v1['Video_ID']=='SSj__PzxOu4') |
                            (df_videos_cleaned_v1['Video_ID']=='dEBmJ1XsMYI') |
                            (df_videos_cleaned_v1['Video_ID']=='1PZkupf3Y7k') |
                            (df_videos_cleaned_v1['Video_ID']=='zkCg8p1HeKc') ].index

df_videos_cleaned_v2 = df_videos_cleaned_v1.drop(rows_to_drop).reset_index(drop=True)

### Grab only the text from the Transcript column (each cell contains a dictionary of text, start time, and duration)

In [8]:
df_videos_cleaned_v3 = grab_transcript_text(df_videos_cleaned_v2)

### Save the cleaned dataframe into a CSV file

In [12]:
df_to_csv('df_videos_cleaned_v3')